# 도서 정보 추출

In [16]:
import json
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

base_dir = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
# base_dir = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
data_dir_org = os.path.join(base_dir, 'data', 'ORIGINAL', '1C')
data_dir = os.path.join(base_dir, 'data', 'FINAL', '2C_0902')

analysis = {1:'1차 분석', 2:'2차 분석', 3: '3차 분석'}
buy = {1:'1차 구매', 2:'2차 구매', 3: '3차 구매'}

In [17]:
i = 2
excel_analy = pd.read_excel(os.path.join(base_dir, '도서목록_전체통합.xlsx'), sheet_name=analysis[i], header=3)[['관리번호', 'ISBN', '도서명','분류']]
excel_buy = pd.read_excel(os.path.join(base_dir, '도서목록_전체통합.xlsx'), sheet_name=buy[i], header=4)[['ISBN', '도서명', '출판일', '코퍼스 1분류', '코퍼스 2분류', '비고']]
excel_buy.fillna("", inplace=True)

## Lv2 파일명 변경, Lv3/4/5 폴더 분류

In [63]:
import shutil
Lv2_isbn_id = excel_analy[excel_analy['분류'] == 'Lv2']
Lv3_isbn_id = excel_analy[excel_analy['분류'] == 'Lv3/4']
Lv5_isbn_id = excel_analy[excel_analy['분류'] == 'Lv5']

In [ ]:
Lv2_isbn_id

In [20]:
# Lv2
# 폴더 만들어서 옮기기
for id in Lv2_isbn_id['관리번호']:
    id = str(id)
    try:
        os.mkdir(os.path.join(data_dir, 'Lv2', id))
        os.rename(os.path.join(data_dir, 'Lv2', id+'.json'), os.path.join(data_dir, 'Lv2', id, id+'.json'))
    except:
        pass

In [ ]:
# for (isbn, id) in zip(cycle1_isbn_id['ISBN'], cycle1_isbn_id['관리번호']):
#     # print(isbn, id)
#     # 복사할 파일 경로
#     source_file = os.path.join(data_dir, '1C_0902', str(isbn)+'.json')
#     # 복사할 목적지 경로
#     destination_path = os.path.join(data_dir, '1C_0902')
#     # 이름변경
#     shutil.copy2(source_file, os.path.join(destination_path, str(id)+".json"))

In [15]:
# Lv분류
lv34 = os.path.join(data_dir, "Lv3_4")
lv5 = os.path.join(data_dir, "Lv5")

for id in os.listdir(data_dir):
    if os.path.splitext(id)[0] in Lv3_isbn_id['관리번호'].astype(str).tolist():
        shutil.move(os.path.join(data_dir, str(id)), lv34)
    elif os.path.splitext(id)[0] in Lv5_isbn_id['관리번호'].astype(str).tolist():
        shutil.move(os.path.join(data_dir, str(id)), lv5)

In [ ]:
os.path.splitext(id)[0] in Lv5_isbn_id['관리번호'].astype(str).tolist()

In [ ]:
id in Lv3_isbn_id['관리번호'].astype(str).tolist()
# print(id)

## Lv2 전체 형식 변경

In [65]:
merge_excel = pd.merge(excel_analy[excel_analy['분류'] == 'Lv2'], excel_buy, on=['ISBN', '도서명'], how='inner')

In [ ]:
merge_excel = merge_excel[['관리번호', 'ISBN', '도서명', '출판일', '코퍼스 1분류', '코퍼스 2분류', '비고']].set_index('관리번호')
merge_excel.head()

In [67]:
data_dir_2 = os.path.join(data_dir, 'Lv2')

In [ ]:
import json
import os

for id in os.listdir(data_dir_2):
    if id.startswith('.DS'):
        continue
    # id = 'SS0299'
    name = str(id)+'.json'
    # origin_name = str(merge_excel.loc[id, 'ISBN'])+'.json'
    origin = json.load(open(os.path.join(data_dir_2, str(id), name), 'r', encoding='utf-8'))
    revision = {
        'file_id': str(merge_excel.loc[id, 'ISBN']),
        'title': merge_excel.loc[id, '도서명'],
        'cat1_domain': merge_excel.loc[id, '코퍼스 1분류'],
        'cat2_sub': merge_excel.loc[id, '코퍼스 2분류'],
        'cat3_specific': merge_excel.loc[id, '비고'],
        'pub_date': str(merge_excel.loc[id, '출판일'])[:10],
        'contents': [],
    }
    for i in range(len(origin)):
        if len(origin[i]['content']) > 0:
            contents_base = {
            'page': f"{int(origin[i]['page']):04d}",
            'chapter': "",
            'page_contents': origin[i]['content'],
                "add_info": []
            }
            revision['contents'].append(contents_base)
    
    json.dump(revision, open(os.path.join(data_dir_2, str(id), name), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    # break

In [ ]:
id

## Lv3/4/5 분류 이름 변경
코퍼스_1분류, 코퍼스_2분류, 비고, 출판일 -> cat1_domain, cat2_sub, cat3_specific, pub_date

In [ ]:
merge_excel = pd.merge(excel_analy[excel_analy['분류'] == 'Lv5'], excel_buy, on=['ISBN', '도서명'], how='inner')
merge_excel = merge_excel[['관리번호', 'ISBN', '도서명', '출판일', '코퍼스 1분류', '코퍼스 2분류', '비고']].set_index('관리번호')
merge_excel.head()

In [ ]:
# merg=merge_excel[merge_excel['코퍼스 2분류'] != '수험서']
merg = merge_excel
merg.head()

In [ ]:
data_dir

In [19]:
# # Lv분류
# # lv34 = os.path.join(data_dir, "Lv3_4")
# lv5 = os.path.join(data_dir, "Lv5")

# for id in merg.index:
#     id = str(id)
#     origin = json.load(open(os.path.join(lv5, id+'_workbook', id+".json"), 'r', encoding='utf-8'))
#     try:
#         origin['cat1_domain']
#     except:
#         revision = {
#             'file_id': origin['file_id'],
#             'title': origin['title'],
#             'cat1_domain': origin['코퍼스_1분류'],
#             'cat2_sub': origin['코퍼스_2분류'],
#             'cat3_specific': origin['비고'],
#             'pub_date': origin['출판일'],
#             'contents': origin['contents']
#         }
    
#         json.dump(revision, open(os.path.join(lv5, id+"_workbook", id+"_name.json"), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [ ]:
290155840 in merg.index

---

# 개별 처리
- 불필요 페이지 제거
- 오타
- 페이지 머리말/꼬리말

In [ ]:
import re, json, os

base_dir = "data/FINAL/2C_0902"
data_dir = os.path.join(base_dir,'Lv2')

# lv2
# name = sorted(os.listdir(data_dir))[3]

# lv3/4
names = []
for n in sorted(os.listdir(data_dir)):
    if n.endswith('workbook'):
        names.append(n.split('_')[0])

name = names[0].split('_')[0]
# origin = json.load(open(os.path.join(data_dir, name, name+'.json'), 'r', encoding='utf-8'))
origin = json.load(open(os.path.join(data_dir, name+"_workbook", name+'.json'), 'r', encoding='utf-8'))
name

### 1차 처리

In [19]:
def n_split(txt, sep):
    result = re.sub(r'(?<![.?!①②③④⑤\[\]])\n(?!\n)', ' ', txt)
    return result.split(sep)

def remove_enter(txt):
    # 문장 내 엔터 처리 (안함)
    return re.sub(r'(?<![.?!\]])\n(?!\n)', ' ', txt)

def extract_options(txt):
    pattern = r'([①②③④⑤]\s*[^①②③④⑤]*)'
    options = re.findall(pattern, txt)
    return [opt.replace("\n", " ").strip() for opt in options if opt.strip()]

def replace_number(number):
    circle_numbers = {'1': '①', '2': '②', '3': '③', '4': '④', '5': '⑤'}
    return circle_numbers[str(number)]

In [20]:
def extract_chapter(c, n):
    #  chapter 추출하기
    page = int(c['page']) - n
    regex = fr"^(.*)\n{page}\n"

    chapter = re.findall(regex, c['page_contents'])

    c['chapter'] = chapter[0].strip()
    c['page_contents'] = re.sub(regex, "", c['page_contents'])
    
    # 248779244.json
    # c['page_contents'] = re.sub(fr"^{page}[가-힇]?", "", c['page_contents'])
    # c['page_contents'] = re.sub(title+str(page)+'\n', "", c['page_contents'])
    return c

def fill_chapter(c):
    # chapter 채우기
    if (c['chapter'] == "") and i >= 1:
        c['chapter'] = origin['contents'][i-1]['chapter']

    # 284618539.json
    # if c['page_contents'] == "": break
    
    # if (c['chapter'] == "") and i >= 1:
    #     c['chapter'] = origin['contents'][i-1]['chapter']
    return c

In [21]:
def extract_qna(c):
    print('---------------------')
    print(c['page'])
# for i in range(len())
    # 문제추출
    # qn_re = r'\[문제\d+\]\s'
    qn_re = r'\s?문제 [0-9]+:'
    # qn_re = r'\s?[0-9][0-9]'
    # info_re = r'\[\w+[.]?\d+\w\]\s'
    # q_re = r"\s*[^?]*\?"
    # ans_re = r"\[정답\]"
    ans_re = r"\s?정답:"
    # exp_re = r"\s\[해설\]"
    exp_re = r"\s?해설:"
    option_re = r"\s?①"
    
    base_add_info = {
                        'tag': "",
                        'type': "question",
                        "description": {
                            "number": "",
                            "question": "",
                            "options": [],
                            'answer' : "",
                            "explanation": ""
                        },
                        "caption":[],
                        "file_path": 0,
                        "bbox": 0
                        }

    # 문제는 있음
    if re.search(qn_re, c['page_contents']) is not None:

        base_add_info['tag'] = f"q_{c['page']}_0001"

        start = re.search(qn_re, c['page_contents']).span()[0]
        try:
            end = re.search(ans_re, c['page_contents']).span()[1]
            # 한 페이지에 문제~정답 모두 있음
            # qna = [qa for qa in re.split(fr"({qn_re}|{info_re}|{q_re}|{exp_re}|{ans_re})", c['page_contents'][start:]) if qa !=""]
            qna = [qa for qa in re.split(fr"({qn_re}|{option_re}|{exp_re}|{ans_re})", c['page_contents'][start:]) if qa !=""]
            # 태그처리
            c['page_contents'] = c['page_contents'].replace(c['page_contents'][start:end+2], "\n{"+f"q_{c['page']}_0001"+"}")
        
        except:
            # 다음페이지 살펴보기
            c2 = origin['contents'][i+1]
            if re.search(ans_re, c2['page_contents']) is not None:
                end = re.search(ans_re, c2['page_contents']).span()[1]

                qna = [qa 
                    # for qa in re.split(fr"({qn_re}|{info_re}|{q_re}|{exp_re}|{ans_re})", 
                    for qa in re.split(fr"({qn_re}|{option_re}|{exp_re}|{ans_re})", 
                                        c['page_contents'][start:]+c2['page_contents'][:end+2])
                    if qa !=""
                    ]
                
                # 태그처리
                c['page_contents'] = c['page_contents'].replace(c['page_contents'][start:], "\n{"+f"q_{c['page']}_0001"+"}")
                c2['page_contents'] = c2['page_contents'].replace(c2['page_contents'][:end+2], "")
            
            # 그 다음 페이지도 살펴보기
            elif re.search(ans_re, c2['page_contents']) is None:
                c3 = origin['contents'][i+2]
                if re.search(ans_re, c3['page_contents']) is not None:
                    end = re.search(ans_re, c3['page_contents']).span()[1]
                    qna = [qa 
                        # for qa in re.split(fr"({qn_re}|{info_re}|{q_re}|{exp_re}|{ans_re})", 
                        for qa in re.split(fr"({qn_re}|{option_re}|{exp_re}|{ans_re})", 
                                            c['page_contents'][start:]+c2['page_contents']+c3['page_contents'][:end+2])
                        if qa !=""
                        ]
                    
                    # 태그처리
                    c['page_contents'] = c['page_contents'].replace(c['page_contents'][start:], "\n{"+f"q_{c['page']}_0001"+"}")
                    c2['page_contents'] = c2['page_contents'].replace(c2['page_contents'], "")
                    c3['page_contents'] = c3['page_contents'].replace(c3['page_contents'][:end+2], "")
            else:
                print("못찾겟다..")

        print(qna)
        for x in range(len(qna)):
            if re.search(qn_re, qna[x]) is not None:
                # number = re.search(r'\[문제(\d+)*\]', qna[x]).group(1)
                number = qna[x].strip()
                print('number:', number)
                # if int(number) < 10:
                #     number = f"{int(number):02}"
                base_add_info['description']['number'] = number
                try:
                    question = qna[x+1]
                    question = question.strip()

                    base_add_info['description']['question'] = question
                except:
                    print("(ERROR) question:", question)

            # if re.search(info_re, qna[x]) is not None:
            #     base_add_info['caption'].append(re.search(r'\[(\w+[.]?\d+\w)\]', qna[x]).group(1))
            #     if base_add_info['caption'][0].startswith("문제"):
            #         base_add_info['caption'].pop(0)

            # if re.search(q_re, qna[x]) is not None:
            #     question = re.search(r'\s*([^?]*\?)', qna[x]).group(1)
            #     options = [ca.strip() for ca in n_split(qna[x+1].replace("❑",""), "\n") if ca != ""]
            #     for ct in range(len(options)):
            #         if '①' not in options[0]:
            #             question += " "+options[0]
                        # options = options[1:]
                
            if re.search(option_re, qna[x]) is not None:
                try:
                    options = [qna[x]+qna[x+1]]
                except:
                    print("(ERROR) options:", options)
                if len(options) == 1:
                    options = extract_options(options[0])
                print("options_re:", options)
                base_add_info['description']['options'] = options

            if re.search(exp_re, qna[x]) is not None:
                explanation = qna[x+1].strip()
                print("explanation:", explanation)
                base_add_info['description']['explanation'] = explanation

            if re.search(ans_re, qna[x]) is not None:
                answer = qna[x+1].strip()
                print("answer:", answer)
                if answer in "①②③④⑤":
                    base_add_info['description']['answer'] = answer
                elif answer.isnumeric():
                    answer = replace_number(answer)
                    base_add_info['description']['answer'] = answer
                else:
                    print(c['page'], end, answer)
            
            # base_add_info['caption'].append("매경 TEST 기출 문제")

        c['add_info'].append(base_add_info)
    return c

In [22]:
def extract_footnote(c):
    # 각주 처리
    for fn in range(1, 43):
        if f"\n{fn} " in c['page_contents']:
            start = c['page_contents'].find(f"\n{fn} ")
            tag = f"note_{c['page']}_{len(c['add_info'])+1:04}"

            c['add_info'].append(
                {
                    "tag": tag,
                    "type": "footnote",
                    "description": c['page_contents'][start+1:], # 두개 겹쳐있으면 그대로..
                    "caption": 0,
                    "file_path": 0,
                    "bbox": 0
                }
            )
            c['page_contents'] = c['page_contents'].replace(c['page_contents'][start:], "{"+tag+"}")
    return c

## 전체 돌리기

In [1]:
import os
import json
from glob import glob

base_dir = "/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar"
data_dir = os.path.join(base_dir, 'data', 'FINAL', '2C_0902')

json_files = []
for root, _, files in os.walk(data_dir):
    for f in files:
        if f.endswith(".json") and ('_' not in f) and ('_workbook' in root):
        #  and ('Lv5' not in root):
            json_files.append(os.path.join(root, f))
            # json_files.append(int(os.path.splitext(f)[0]))

In [ ]:
# origin = json.load(open(json_files[0], 'r', encoding='utf-8'))

In [2]:
json_files = sorted(json_files)
json_files

['/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0119_workbook/SS0119.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0121_workbook/SS0121.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0122_workbook/SS0122.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0124_workbook/SS0124.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0125_workbook/SS0125.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0126_workbook/SS0126.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0127_workbook/SS0127.json',
 '/Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL/2C_0902/Lv2/SS0128_workbook/SS0128.json',
 '/Users/jinym/L

In [4]:
import tools.process_quizbook as quiz

quiz.main(json_files[2])

In [ ]:
# for j in json_files:
if True:
    j = json_files[1]
    origin = json.load(open(j, 'r', encoding='utf-8'))
    
    new = {
        'file_id': origin['file_id'],
        'title': origin['title'],
        'cat1_domain': origin['cat1_domain'],
        'cat2_sub': origin['cat2_sub'],
        'cat3_specific': origin['cat3_specific'],
        'pub_date': origin['pub_date'],
        'contents': [],
    }

    for i in range(len(origin['contents'])):
        contents = origin['contents'][i]
        
        # c = extract_qna(contents)
        # page_contents = remove_enter(contents['page_contents'])
        # contents['page_contents'] = page_contents
        c = fill_chapter(contents)
        # c = contents
        # print(contents)
        
        if len(c['page_contents']) > 0:
            new['contents'].append(c)
        

    json.dump(new, open(j, 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    # json.dump(new, open(j.replace(".json", "_new.json"), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    # json.dump(new, open(os.path.join(data_dir, name+'_workbook', name+'.json'), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)
    

In [ ]:
for i, j in enumerate(json_files):
    if '276709608' in j:
        print(i, j)

### 2차 처리

In [ ]:
import re, json, os

base_dir = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL"
data_dir = os.path.join(base_dir, '1C_0902', 'Lv2')

name = sorted(os.listdir(data_dir))[0]

In [ ]:
origin = json.load(open(os.path.join(data_dir, name, name+'.json'), 'r', encoding='utf-8'))
name

In [ ]:
# 5분 손해사정사 제3보험 (321494387) - 기출/답안 추출
new = {
    'file_id': origin['file_id'],
    'title': origin['title'],
    'cat1_domain': origin['cat1_domain'],
    'cat2_sub': origin['cat2_sub'],
    'cat3_specific': origin['cat3_specific'],
    'pub_date': origin['pub_date'],
    'contents': [],
}


for i in range(len(origin['contents'])):
    # i = 126
    c = origin['contents'][i]

    base_add_info = {
    'tag': "",
    'type': "question",
    "description": {
        "number": "",
        "question": "",
        "options": 0,
        'answer' : "",
        "explanation": ""
    },
    "caption":[],
    "file_path": 0,
    "bbox": 0
    }
    # 문제추출
    q_re = r'\[[0-9]{4} 기출\]'
    ans_re = r'\[[0-9]{4} 답안\]'
    key_re = r'\s키워드'

    # break
    try:
    # if 1:
        # [기출] ~ [답안]꼴
        q_start = re.search(q_re, c['page_contents']).span()[0]
        q_end = re.search(ans_re, c['page_contents'][q_start:]).span()[0] # 그꼴이 아니면 여기서 에러남
        a_start = q_end + q_start
        
        question = c['page_contents'][q_start:q_start+q_end]

       
        # caption 달기
        caption = re.findall(r'\[([0-9]{4} 기출)\]', question)[0]
        base_add_info['caption'].append(caption)

        question = re.sub(q_re, "", question).strip()      

         # 문제번호 뽑기
        number = re.findall(r"\D?(\d+)\D?\s", question)[0]
        base_add_info['description']['number'] = number.strip()
        question = re.sub(r"\D?\d+\D?\s", "", question).strip()


        # 태그 처리
        tag = f"q_{c['page']}_0001"
        base_add_info['tag'] = tag

        
                
        # 한 페이지에 [답안] 없으면
        if re.search(key_re, c['page_contents'][a_start:]) is None:
            print("in two pages")
            c2 = origin['contents'][i+1]
            a_end = re.search(key_re, c2['page_contents']).span()[0]
            # print(c2)
            answer = c['page_contents'][a_start:] + '\n' + c2['page_contents'][:a_end]

            c['page_contents'] += '\n' + c2['page_contents'][:a_end]
            c2['page_contents'] = c2['page_contents'].replace(c2['page_contents'][:a_end], "")

            c['page_contents'] = c['page_contents'].replace(c['page_contents'][q_start:], "{"+tag+"}")

            if c2['page_contents'].strip().startswith('키워드'):
                keyword_org = c2['page_contents'].strip().split("\n")[0]
                keyword = keyword_org.replace("키워드", "키워드: ").replace("  ", " ")
                base_add_info['caption'].append(keyword)
                c2['page_contents'] = c2['page_contents'].replace("\n"+keyword_org, "")
            # print(c2)
        # 한 페이지에 [답안]까지 있음
        else:
            print("in one page")
            a_end = re.search(key_re, c['page_contents'][a_start:]).span()[0]
            answer = c['page_contents'][a_start:a_start+a_end]
            if c['page_contents'][a_start+a_end:].strip().startswith('키워드'):
                keyword_org = c['page_contents'][a_start+a_end:].strip().split("\n")[0]
                keyword = keyword_org.replace("키워드", "키워드: ").replace("  ", " ")
                base_add_info['caption'].append(keyword)

            c['page_contents'] = c['page_contents'].replace(c['page_contents'][q_start:a_start+a_end], "{"+tag+"}")
            c['page_contents'] = c['page_contents'].replace("\n"+keyword_org, "")

        answer = re.sub(ans_re, "", answer)
        

        base_add_info['description']['question'] = question.strip()
        base_add_info['description']['answer'] = answer.strip()

        c['add_info'].append(base_add_info)

    except Exception as e:
    # else:
        if re.search(q_re, c['page_contents']) is not None:
            print(c['page'])
        else:
            # print(e, c['page'])
            pass

    # break
    if len(c['page_contents']) > 0:
        new['contents'].append(c)
    



    # break

# json.dump(origin, open(os.path.join(data_dir, name, name+'_new.json'), 'w', encoding='utf-8'), ensure_ascii=False, indent=4)


In [ ]:
exp = [
  "◆ ① (O) [상법 제317조 제2항 제8호, 제9호]",
                            "◆ ② (O) [상법 제342조의2 제1항]",
                            "◆ ③ (O) [상법 제382조 제3항 제1호]",
                            "◆ ④ (X) 두지 못한다. [상법 제408조의2 제1항] / ⑤ (O) [상법 제408조의2 제2항]",
                            "✓ 제408조의2(집행임원 설치회사, 집행임원과 회사의 관계) ① 회사는 집행임원을 둘 수 있다. 이 경우 집행임원을 둔 회사(이하 \"집행임원 설치회사\"라 한다) 는 대표이사를 두지 못한다.",
                            "✓ ② 집행임원 설치회사와 집행임원의 관계는 「민법」 중 위임에 관한 규정을 준용한다."
]

exp = "\n".join(exp).replace('◆ ', "")
exp

---
# add_info
```json
{
    "tag": "q_0000_0001",
    "type": "question",
    "description": {
        "number": "",
        "question": "",
        "options": [],
        "options": null,
        "answer": "",
        "explanation": ""
    },
    "caption": [
        " 기출",
        "키워드: "
    ],
    "file_path": null,
    "bbox": null
},
{
    "tag": "note_0000_0001",
    "type": "footnote",
    "description": "1 이러한 계약을 법적으로는 금전소비대차계약이라고 한다. 차용증서를 영어로는 I owe you.의 소리를 따라 IOU 라고 한다.",
    "caption": null,
    "file_path": null,
    "bbox": null
},
{
    "tag": "tb_0000_0001",
    "type": "table",
    "description": "\\begin{tabular}{|c|c|c|c|}\n\\hline\n나라 이름 & 지수 이름 & 포괄 종목 & 작성 기관 \\\\\\hline\n\\multirow{2}{*}{한 국} & KOSPI & \\makecell[l]{유가증권시장 상장\\\\전종목} & 한국거래소 \\\\\\cline{2-4}\n& KOSDAQ & \\makecell[l]{코스닥 상장 전종목} & 한국거래소 \\\\\\hline\n\\multirow{4}{*}{미 국} & DJIA & \\makecell[l]{뉴욕거래소, 나스닥\\\\상장 30 개 종목} & 다우존스사 \\\\\\cline{2-4}\n& S\\&P 500 & \\makecell[l]{뉴욕거래소,\\\\미국거래소, 나스닥\\\\상장 500 개 종목} & \\makecell[c]{Standard \\&\\\\Poors 사} \\\\\\hline\n\\multirow{3}{*}{일 본} & TOPIX & \\makecell[l]{동경거래소 상장\\\\전종목} & 동경거래소 \\\\\\cline{2-4}\n& NIKKEI 225 & \\makecell[l]{동경거래소 상장\\\\225 개 종목} & 일본경제신문사 \\\\\\hline\n영 국 & FTSE 100 & \\makecell[l]{런던거래소 상장\\\\100 개 종목} & FTSE \\\\\\hline\n독 일 & DAX 30 & \\makecell[l]{프랑크푸르트 거래소\\\\상장 30 개 종목} & Deutsche Boerse \\\\\\hline\n프랑스 & CAC 40 & \\makecell[l]{Euronext Paris 상장\\\\40 개 종목} & Euronext \\\\\\hline\n중 국 & 상해종합지수 & \\makecell[l]{상해거래소 상장\\\\전종목} & 상해거래소 \\\\\\hline\n\\end{tabular}",
    "caption": [
        "표2. 주가지수 개요"
    ],
    "file_path": "??????/crop/tb_0000_0001.png",
    "bbox": null
},
{
    "tag": "img_0000_0001",
    "type": "image",
    "description": null,
    "caption": [
        "도표1: 경제주체간 상품, 생산요소 및 자금의 흐름"
    ],
    "file_path": "?????/crop/img_0000_0001.png",
    "bbox": null
}

```


In [ ]:
import re, json, os

base_dir = "/Users/yejin/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/FINAL"
data_dir = os.path.join(base_dir, '1C_0902', 'Lv2')

json_files = []
for root, _, files in os.walk(data_dir):
    for f in files:
        if f.endswith(".json"):
            json_files.append(os.path.join(root, f))

# json_files

In [ ]:
for js in json_files:
    origin = json.load(open(js, 'r', encoding='utf-8'))
    if str(origin).find('": 0,') != -1:
        print(js)
        new = str(origin)
        new = new.replace('": 0,', '": null,')
        json.dump(new, open(js, 'w', encoding='utf-8'), ensure_ascii=False, indent=4)

In [ ]:
str(origin).find(": 0,")

In [ ]:
new = str(origin)
print(new[791:800])
new = new.replace(": 0,", ": null,")
print(new[791:800])

In [ ]:
name